<a href="https://colab.research.google.com/github/tmyok/Colaboratory/blob/master/pytorch_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch

In [25]:
print(torch.__version__)
print("cuda =",torch.cuda.is_available())

1.5.0+cu101
cuda = True


In [0]:
use_cuda = torch.cuda.is_available()
train_batch_size = 64
test_batch_size = 1000
lr = 1.0
gamma = 0.7
epochs = 10
log_interval = 10

In [0]:
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [0]:
from torchvision import datasets, transforms
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                  transform=transforms.Compose([
                  transforms.ToTensor(),
                  transforms.Normalize((0.1307,), (0.3081,))
                  ])),
    batch_size=train_batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                  transforms.ToTensor(),
                  transforms.Normalize((0.1307,), (0.3081,))
                  ])),
    batch_size=test_batch_size, shuffle=True, **kwargs)

In [0]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [0]:
model = Net().to(device)
import torch.optim as optim
optimizer = optim.Adadelta(model.parameters(), lr=lr)

In [0]:
def train(model, device, train_loader, optimizer, epoch, log_interval):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [32]:
from torch.optim.lr_scheduler import StepLR
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch, log_interval)
    test(model, device, test_loader)
    scheduler.step()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.307846
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.539303
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.781088
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.487064
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.228497
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.408723
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.466630
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.156251
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.152584
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.321903
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.250667
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.519037
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.065673
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.266310
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.280805
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.185145
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.427332
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.336705
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.083794
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.183890
Train Epoch: 1 [12800/60000 (